In [15]:
import tempfile

from monai.visualize.utils import matshow3d
from monai.data import CacheDataset, ThreadDataLoader, PersistentDataset
from monai.transforms import (
    Compose,
    LoadImaged,
    MapTransform,
    ScaleIntensityRanged,
    Orientationd,
    Spacingd,
    CropForegroundd,
    SpatialCropd,
    RandFlipd,
    RandRotate90d,
    RandShiftIntensityd,
    EnsureTyped,
    BorderPad,
    SaveImaged,
    Lambdad,
)

from monai.data import  decollate_batch

import matplotlib.pyplot as plt
from monai.networks.nets import SwinUNETR

from monai.losses import DiceCELoss

from monai.metrics import DiceMetric, ROCAUCMetric

from monai.transforms import AsDiscrete

from tqdm import tqdm

from nodule_transforms.LoadArterysMaskd import LoadArterysMaskd
from nodule_transforms.AddCustomerMetadatad import AddCustomerMetadatad
from nodule_transforms.CropToNoduled import CropToNoduled



import os
import torch



from data_tools.datatools import load_cancer_nodules_datalist, split_data


directory = './monai_data'
root_dir = tempfile.mkdtemp() if directory is None else directory
cache_dir = './training_cache'

CANCER_NODULES_DATASET_DIR = '/media/vlermakov/data/UCSDNodules/Metastatic/'
PRE_PROCESSED_DIRECTORY = '/home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/'


datalist = load_cancer_nodules_datalist(CANCER_NODULES_DATASET_DIR, PRE_PROCESSED_DIRECTORY)

Processing MRN 211 Phonetic_ID Fesaho
Processing MRN 210 Phonetic_ID Quabemo
Processing MRN 209 Phonetic_ID Quekelurn
Processing MRN 208 Phonetic_ID Jufueklip
Processing MRN 207 Phonetic_ID Moyabdof
Processing MRN 206 Phonetic_ID Tudipi
Processing MRN 205 Phonetic_ID Nousisig
Processing MRN 204 Phonetic_ID Budueyi
Processing MRN 203 Phonetic_ID Jenuda
Processing MRN 202 Phonetic_ID Cretaha
Processing MRN 201 Phonetic_ID Pejaru
Processing MRN 200 Phonetic_ID Barethue
Processing MRN 199 Phonetic_ID Miekefre
Processing MRN 198 Phonetic_ID Labfaljon
Processing MRN 196 Phonetic_ID Togepel
Processing MRN 195 Phonetic_ID Diclama
Processing MRN 194 Phonetic_ID Woroxof
Processing MRN 193 Phonetic_ID Loomako
Processing MRN 192 Phonetic_ID Puguxiep
Processing MRN 191 Phonetic_ID Alnpatik
Processing MRN 190 Phonetic_ID Beelelu
Processing MRN 189 Phonetic_ID Omlolit
Processing MRN 189 Phonetic_ID Pihoko
Processing MRN 189 Phonetic_ID Paparser
Processing MRN 187 Phonetic_ID Huciku
Processing MRN 187

Processing MRN 27 Phonetic_ID Hehuji
Processing MRN 26 Phonetic_ID Frequalad
Processing MRN 25 Phonetic_ID Nuborif
Processing MRN 24 Phonetic_ID Figooso
Processing MRN 23 Phonetic_ID Hilescuet
Processing MRN 22 Phonetic_ID Fabacou
Processing MRN 21 Phonetic_ID Pokuemo
Processing MRN 20 Phonetic_ID Sracaqua
Processing MRN 19 Phonetic_ID Hakingques
Processing MRN 18 Phonetic_ID Tuerekom
Processing MRN 17 Phonetic_ID Swofeku
Processing MRN 16 Phonetic_ID Ridine
Processing MRN 15 Phonetic_ID Mohuteg
Patch ID nan is not a string
Processing MRN 14 Phonetic_ID Rakajid
Processing MRN 13 Phonetic_ID Jaquesif
Processing MRN 12 Phonetic_ID Bekubo
Processing MRN 11 Phonetic_ID Agunit
Processing MRN 10 Phonetic_ID Nidogek
Processing MRN 9 Phonetic_ID Lorutink
Processing MRN 9 Phonetic_ID Dihudi
Processing MRN 8 Phonetic_ID Socaba
Processing MRN 7 Phonetic_ID Ukebjiem
Processing MRN 6 Phonetic_ID Olcisof
Processing MRN 6 Phonetic_ID Jebeemu
Processing MRN 5 Phonetic_ID Jasnuflo
Processing MRN 4 Phon

In [8]:
import wandb
import pandas as pd

WANDB_PROJECT_NAME = "NoduleClassification"

#87e3b4f079c5a73e70e9ebd3bda9dc48ee553d9f
wandb.login(key='87e3b4f079c5a73e70e9ebd3bda9dc48ee553d9f')

run_parameters = {
    "dataset_folder": CANCER_NODULES_DATASET_DIR,
    "pre_processed_folder": PRE_PROCESSED_DIRECTORY,
    "resample_resolution": 0.75,
    "intensity_range_min": -1000,
    "intensity_range_max": 400
}

run = wandb.init(
        # Set the project where this run will be logged
        project=WANDB_PROJECT_NAME,
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=f"PreprocessData",
        job_type="dataset",
        # Track hyperparameters and run metadata
        config=run_parameters
        )



dataset_table = wandb.Table(dataframe=pd.DataFrame(datalist))

dataset_artifact = wandb.Artifact(
    'datalist_table',
    type="dataset"
    )

dataset_artifact.add(dataset_table, "datalist_table")

# Log the raw csv file within an artifact to preserve our data
dataset_artifact.add_file(os.path.join(run.config.dataset_folder, 'dataset.csv'))

# Log the table to visualize with a run...
run.log({"dataset_table": dataset_table})
run.log_artifact(dataset_artifact)




wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


<Artifact datalist_table>

In [22]:
processed_scans = []
unprocessed_scans = []
n_processed = 0
for p in datalist:
    if(p['image'] == '/media/vlermakov/data/UCSDNodules/Metastatic/data/Woroxof__2.25.260876099222205460618271003114894681227/2.25.260876099222205460618271003114894681227/2.25.314316397946104988922756588776690923298'):
        print("Skipping this image.")
        continue
    if p['pre_processed']:
        n_processed += 1 
        processed_scans.append(p)
    else:
        unprocessed_scans.append(p)
print(len(unprocessed_scans))

Skipping this image.
573


{'mask': '/media/vlermakov/data/UCSDNodules/Metastatic/segmentation/Fesaho-1697064513883.json',
 'image': '/media/vlermakov/data/UCSDNodules/Metastatic/data/Fesaho__2.25.303573916076522530325854521132938747310/2.25.303573916076522530325854521132938747310/2.25.23160264776699173714896903404928629326',
 'label': 2,
 'phonetic_id': 'Fesaho',
 'patch_id': '652215cc536992ecbb15f106-69c35cad0a8d3c48-1',
 'mrn': '211',
 'pre_processed': False}

In [ ]:
train_data_list, val_data_list = split_data(datalist)

In [ ]:
def get_data_list_stats(data_list):
    n_pos = 0
    n_neg = 0
    n_total = 0
    for l in data_list:
        if(l['label'] == 0):
            n_neg +=1
        elif(l['label'] == 1):
            n_pos += 1
            
        n_total += 1
        
    print(f"P:{n_pos} N:{n_neg} T:{n_total} %pos: {n_pos/n_total:.2f}")



In [ ]:
get_data_list_stats(train_data_list)

In [ ]:
get_data_list_stats(val_data_list)

In [23]:
def file_name_formatter(metadict: dict, saver) -> dict:
    return {"subject": metadict['phonetic_id'] + "__" + metadict['patch_id'], "idx": 0}
    
    
if(len(unprocessed_scans) > 0):
    pre_process_transforms = Compose(
        [
            LoadImaged(keys=["image"], ensure_channel_first=True, image_only=False),
            LoadArterysMaskd(keys=["mask"], ensure_channel_first=True),
            AddCustomerMetadatad(keys=["image","mask"],meta_keys=["mrn","phonetic_id","patch_id"]),
            ScaleIntensityRanged(
                keys=["image"],
                a_min=run.config.intensity_range_min,
                a_max=run.config.intensity_range_max,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
            Spacingd(
                keys=["image","mask"],
                pixdim=(run.config.resample_resolution, run.config.resample_resolution, run.config.resample_resolution),
                mode=("bilinear","nearest"),
            ),
            CropToNoduled(keys=["image","mask"]),
            SaveImaged(keys=["image"], output_name_formatter=file_name_formatter ,output_dir=PRE_PROCESSED_DIRECTORY, output_postfix="image", output_ext=".nii.gz", resample=False),
            SaveImaged(keys=["mask"], output_name_formatter=file_name_formatter, output_dir=PRE_PROCESSED_DIRECTORY, output_postfix="mask", output_ext=".nii.gz", resample=False)
        ]
    )

    train_ds = CacheDataset(
        data=unprocessed_scans[1:],
        transform=pre_process_transforms,
        cache_num=len(unprocessed_scans[1:]),
        cache_rate=1.0,
        num_workers=8,
    )



Loading dataset:   0%|                                                                                                                                                         | 0/572 [00:00<?, ?it/s]

2023-10-16 00:39:33,994 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Hadaseb__63532b486d8ba20945492871-cea5c802b75e489a-1/Hadaseb__63532b486d8ba20945492871-cea5c802b75e489a-1_image_0.nii.gz
2023-10-16 00:39:34,894 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Hadaseb__63532b486d8ba20945492871-cea5c802b75e489a-1/Hadaseb__63532b486d8ba20945492871-cea5c802b75e489a-1_mask_0.nii.gz
2023-10-16 00:39:47,509 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Coofapas__6355a8a4fa791c099aa28b37-d0c28a26af8e2699-3/Coofapas__6355a8a4fa791c099aa28b37-d0c28a26af8e2699-3_image_0.nii.gz
2023-10-16 00:39:47,988 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fi

Loading dataset:   0%|▎                                                                                                                                              | 1/572 [00:44<7:07:14, 44.89s/it]

2023-10-16 00:40:08,483 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Raswizu__63531192a7e5600a0338d705-a35561487dff4e1c-1/Raswizu__63531192a7e5600a0338d705-a35561487dff4e1c-1_image_0.nii.gz
2023-10-16 00:40:08,766 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Sufourog__635311d4c0271909c1499bc4-abaf860bda383fc8-1/Sufourog__635311d4c0271909c1499bc4-abaf860bda383fc8-1_image_0.nii.gz
2023-10-16 00:40:08,972 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Raswizu__63531192a7e5600a0338d705-a35561487dff4e1c-1/Raswizu__63531192a7e5600a0338d705-a35561487dff4e1c-1_mask_0.nii.gz
2023-10-16 00:40:09,014 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fi

Loading dataset:   1%|▊                                                                                                                                              | 3/572 [01:33<4:24:28, 27.89s/it]

...

Loading dataset:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 564/572 [1:25:52<01:07,  8.42s/it]

2023-10-16 02:05:10,016 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-7/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-7_image_0.nii.gz
2023-10-16 02:05:10,035 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-2/Sopoopra__63979710b75a1a0997e54f40-5f10b4917eccbda0-2_image_0.nii.gz
2023-10-16 02:05:10,107 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-7/Tusliblu__639d0eea65dc5409534a9682-ed7d5ece31464573-7_mask_0.nii.gz
2023-10-16 02:05:10,136 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075m

Loading dataset:  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 567/572 [1:25:54<00:28,  5.71s/it]

2023-10-16 02:05:17,214 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Minfirag__639e23cbea9f2a08f65eef42-7be7b2798d23ebf2-3/Minfirag__639e23cbea9f2a08f65eef42-7be7b2798d23ebf2-3_image_0.nii.gz
2023-10-16 02:05:17,308 INFO image_writer.py:197 - writing: /home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed/Minfirag__639e23cbea9f2a08f65eef42-7be7b2798d23ebf2-3/Minfirag__639e23cbea9f2a08f65eef42-7be7b2798d23ebf2-3_mask_0.nii.gz


Loading dataset: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 572/572 [1:26:02<00:00,  9.02s/it]


In [24]:

nodules_dataset_artifact = wandb.Artifact(
    'nodules_and_masks_dataset',
    type="dataset"
    )

# Log the raw csv file within an artifact to preserve our data
nodules_dataset_artifact.add_dir(run.config.pre_processed_folder)

run.log_artifact(nodules_dataset_artifact)


wandb: Adding directory to artifact (/home/vlermakov/Development/Nodules/Nodule-Detection-LUNA2016/processed_data_full_range_075mm_fixed)... Done. 0.7s


<Artifact nodules_and_masks_dataset>

In [25]:
run.finish()

In [ ]:
fname = '/media/vlermakov/data/UCSDNodules/Metastatic/data/Sukicuk__2.25.113743471811282083585069081717620800044/2.25.113743471811282083585069081717620800044/2.25.326808598846151585390527901137079074435'

In [ ]:
import re


match = re.search(r'/(\w+__)\d+\.\d+\.\d+/\d+\.\d+\.(\d+)/\d+\.\d+\.(\d+)', fname)

if match:
    result = match.group(1) + match.group(3)
    print(result)

In [ ]:
unprocessed_scans